# **1-Dataset Analysis:**


## *i) Cleaning Dataset:* <br />
   Jan 23 Last edits

Latest check 30th of Jan. 2023 at 5:00PM

### **Importings:**

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from datetime import datetime 
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import plotly.express as px
from ast import literal_eval




In [2]:
data= pd.read_csv("../data-history/up-to-date-MAL/anime_Feb23.csv")


print(data.shape)
data.head(1)

(23936, 31)


,id,title,media_type,mean,num_scoring_users,status,num_episodes,start_date,end_date,source,...,studios,synopsis,nsfw,created_at,updated_at,main_picture_medium,main_picture_large,alternative_titles_en,alternative_titles_ja,alternative_titles_synonyms
0,16498,Shingeki no Kyojin,tv,8.53,2585993,finished_airing,25.0,2013-04-07,2013-09-29,manga,...,['Wit Studio'],"Centuries ago, mankind was slaughtered to near...",white,2012-12-05 12:03:21,2023-01-12 15:12:07,https://api-cdn.myanimelist.net/images/anime/1...,https://api-cdn.myanimelist.net/images/anime/1...,Attack on Titan,進撃の巨人,"['AoT', 'SnK']"


In [3]:
data.columns

Index(['id', 'title', 'media_type', 'mean', 'num_scoring_users', 'status',
       'num_episodes', 'start_date', 'end_date', 'source', 'num_list_users',
       'popularity', 'num_favorites', 'rank', 'average_episode_duration',
       'rating', 'start_season_year', 'start_season_season',
       'broadcast_day_of_the_week', 'broadcast_start_time', 'genres',
       'studios', 'synopsis', 'nsfw', 'created_at', 'updated_at',
       'main_picture_medium', 'main_picture_large', 'alternative_titles_en',
       'alternative_titles_ja', 'alternative_titles_synonyms'],
      dtype='object')

### Taking care of nulls and drops:

In [4]:
drops=["main_picture_medium","main_picture_large","broadcast_day_of_the_week","broadcast_start_time","alternative_titles_en","alternative_titles_ja","alternative_titles_synonyms"]
data_main=data.drop(drops,axis=1)

In [5]:
sum(data_main.isnull().sum())

43697

In [6]:
data_main.fillna(value=data['mean'].mean,inplace=True)
data_main.fillna(value=data['rank'].mean,inplace=True)
data_main.fillna(value=data['num_favorites'].mean,inplace=True)

In [7]:
sum(data_main.isnull().sum())


0

In [8]:

data_main.dropna(subset=['mean','source','num_episodes','start_date','end_date','rank','average_episode_duration','rating','start_season_year','synopsis'],inplace=True)
print(sum(data_main.isnull().sum()))
print(data_main.shape)

0
(23936, 24)


In [9]:
data_main.nunique()

id                          23936
title                       23936
media_type                      7
mean                          556
num_scoring_users            8214
status                          3
num_episodes                  252
start_date                   8224
end_date                     8035
source                         17
num_list_users              10784
popularity                  23767
num_favorites                1768
rank                        21633
average_episode_duration     2917
rating                          7
start_season_year             102
start_season_season             5
genres                       5282
studios                      1488
synopsis                    19294
nsfw                            2
created_at                  23936
updated_at                  23903
dtype: int64

In [15]:
data_main.columns

Index(['id', 'title', 'media_type', 'mean', 'num_scoring_users', 'status',
       'num_episodes', 'start_date', 'end_date', 'source', 'num_list_users',
       'popularity', 'num_favorites', 'rank', 'average_episode_duration',
       'rating', 'start_season_year', 'start_season_season', 'genres',
       'studios', 'synopsis', 'nsfw', 'created_at', 'updated_at'],
      dtype='object')

## Encoding and Adjusting Dtypes:
 Using separate Data_Frame for reviewing, Yes, Enough ram is available.

###  NLP Pre-processing

#### **Applying Key-BERT for Keywords extraction:**

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
from tqdm.notebook import tqdm
import ast
import re
import spacy as sp
from keybert import KeyBERT
from sklearn.feature_extraction.text import TfidfVectorizer


In [12]:
data_main.title.head(10)

0                  Shingeki no Kyojin
1                          Death Note
2    Fullmetal Alchemist: Brotherhood
3                       One Punch Man
4                    Sword Art Online
5               Boku no Hero Academia
6                    Kimetsu no Yaiba
7                              Naruto
8                         Tokyo Ghoul
9              Hunter x Hunter (2011)
Name: title, dtype: object

In [13]:
NLP = sp.load("en_core_web_lg")
TITLE = 'Death Note'
key_model = KeyBERT()
data_main = data_main[~data_main.title.duplicated(keep='first')]
text = data_main[data_main['title'] == TITLE].synopsis.values[0]
def clean_text(text):
    text = re.sub("[\(\[].*?[\)\]]", "", text)
    text = text.replace('\n', "").replace('\r', "")
    text = text.replace('', "")
    text = re.sub('[^a-zA-Z]', " ", str(text))
    text = ' '.join(text.split())
    text = text.lower()
    doc = NLP(text)
    return doc

doc = clean_text(text)
print(doc)

brutal murders petty thefts and senseless violence pollute the human world in contrast the realm of death gods is a humdrum unchanging gambling den the ingenious year old japanese student light yagami and sadistic god of death ryuk share one belief their worlds are rotten for his own amusement ryuk drops his death note into the human world light stumbles upon it deeming the first of its rules ridiculous the human whose name is written in this note shall die however the temptation is too great and light experiments by writing a felon s name which disturbingly enacts his first murder aware of the terrifying godlike power that has fallen into his hands light under the alias kira follows his wicked sense of justice with the ultimate goal of cleansing the world of all evil doers the meticulous mastermind detective l is already on his trail but as light s brilliance rivals l s the grand chase for kira turns into an intense battle of wits that can only end when one of them is dead


In [17]:
data_main.loc[:,'cleaned_syn'] = data_main.loc[:,'synopsis'].astype(str).apply(clean_text)

In [27]:
data_main.columns

Index(['id', 'title', 'media_type', 'mean', 'num_scoring_users', 'status',
       'num_episodes', 'start_date', 'end_date', 'source', 'num_list_users',
       'popularity', 'num_favorites', 'rank', 'average_episode_duration',
       'rating', 'start_season_year', 'start_season_season', 'genres',
       'studios', 'synopsis', 'nsfw', 'created_at', 'updated_at',
       'cleaned_syn'],
      dtype='object')

In [29]:
df_string=data_main[['title','synopsis','cleaned_syn']]

In [30]:
df_num=pd.get_dummies(data_main, columns=["media_type","source","nsfw","genres","rating","studios"], prefix=["media_type","source","nsfw","genres","rating","studios"])
df_num[['title','mean','num_scoring_users','num_episodes','popularity','rank']]=data_main[['title','mean','num_scoring_users','num_episodes','popularity','rank']]

In [32]:
df_num.shape,df_string.shape

((23936, 6822), (23936, 3))

In [ ]:
# Based on https://stackoverflow.com/questions/48925328/how-to-get-all-noun-phrases-in-spacy
def get_candidates(doc):
    # code to recursively combine nouns
    # 'We' is actually a pronoun but included in your question
    # hence the token.pos_ == "PRON" part in the last if statement
    # suggest you extract PRON separately like the noun-chunks above

    index = 0
    noun_indices = [i for i, token in enumerate(doc) if token.pos_ == 'NOUN']
    candidates = []
    for idxValue in noun_indices:
        start = doc[idxValue].left_edge.i if not bool(doc[idxValue].left_edge.ent_type_) else idxValue
        finish = doc[idxValue].right_edge.i+1 if not bool(doc[idxValue].right_edge.ent_type_) else idxValue + 1
        if 0 < finish-start < 7:
            span = doc[start:finish]
            candidates.append(span.text)
    return candidates

candidates = get_candidates(doc)
print(candidates)


In [ ]:
key_model = KeyBERT()
keywords = key_model.extract_keywords(doc.text, candidates=candidates, use_mmr=True, diversity=0.5)

print(keywords)

* Creating clean text, nouns and keywords from synopsis.
* Separate in new df for data analysis.
* Delete Syns entries from main df.


In [ ]:
df.loc[:, 'nouns'] = df.loc[:,'cleaned_syn'].apply(get_candidates)

In [ ]:
syns=['id','title','synopsis','cleaned_syn','nouns']
dfsyn=df[syns]
dfsyn['genres']=data_main['genres']
dfsyn.head(10)

In [ ]:
dropssyn=['synopsis','cleaned_syn','nouns']
df=df.drop(dropssyn,axis=1)

In [ ]:
dfsyn['cleaned_syn'].values[3]

In [ ]:
dfsyn.loc[:, 'doc_clean'] = dfsyn.loc[:,'cleaned_syn'].apply(NLP)

In [ ]:
dfsyn['doc_clean'].values[3]

In [ ]:
#dfsyn.drop('cleaned_syn',inplace=True,axis=1)
dfsyn.columns

In [ ]:
#something weird
keyword= [None] * len(dfsyn)
docs= [None] * len(dfsyn)
for i  in range(len(dfsyn)):
    docs[i]= dfsyn["doc_clean"].values[i].text
    nouns = dfsyn["nouns"]
docs[3]

In [ ]:

for i  in range(len(dfsyn)) :   
    keyword[i] = key_model.extract_keywords(docs=docs[i],candidates=nouns.values[i],use_mmr=True, diversity=0.6)
keywordunp= [None]* len(keyword)

In [ ]:
import itertools
for i in range(len(keyword)):

        keywordunp[i]=list(itertools.chain(*keyword[i]))
        
len(keyword),len(keywordunp)

In [ ]:
keywordunp[10]
keywords= [1]* len(keyword)

In [ ]:

for i in range(len(keywordunp)):
    for j in range(len(keyword[i])):
        keywords[i]=keywordunp[i][0::2]

In [ ]:
len(keywords),keywords[3]

In [ ]:
extracted_keywords=np.array(keywords)


In [ ]:
dfsyn['keywords']=extracted_keywords.tolist()
dfsyn['genres']=data_main['genres']
dfsyn['rating']=data_main['rating']
dfsyn['media_type']=data_main['media_type']
dfsyn.head(4)

In [ ]:
dfsyn.columns

In [ ]:
dfsyn.tail(1)

In [ ]:
#dfsynsave=dfsyn.to_csv("../data-history/up-to-date-MAL/anime-synopsis-keywords-nlp.csv",index=False)

## *ii)EDA:*

In [ ]:
data_main.columns

In [ ]:
sns.set_style("dark")
plt.figure(figsize=(12,8))
plt.hist(df['mean'], bins=70,)
plt.show

In [ ]:
fig = px.pie(data_main, 'media_type')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

*Notes:* Naturally TV has higher percentage as anime media.


In [ ]:
corr = data_main.corr()

# Set up the matplotlib plot configuration
#
f, ax = plt.subplots(figsize=(16, 10))
#
# Generate a mask for upper traingle
#
mask = np.triu(np.ones_like(corr, dtype=bool))
#
# Configure a custom diverging colormap
#
cmap = sns.diverging_palette(230, 20, as_cmap=True)
#
# Draw the heatmap
#
sns.heatmap(corr, annot=True, mask = mask, cmap=cmap)

*Notes:*  So, basically interesting factors that are affecting the mean factor are : rank, popularity, num_scoring_users, ignore num_list_users for now till further investigation of difference.

In [ ]:
fig = px.histogram(data_main[data_main['start_date'].dt.year >= 1980], x='start_date', color='media_type')
fig.update_layout(bargap=0.1)

*Notes:* Obviously 2016 was a good year for Otakus :3 specially summer-Autumn-Fall seasons, with 119 tv, 45 movie, 23 ova, 61 ona, 60 special and 41 music. (Gotta check watching list lmao).

In [ ]:
data_main.groupby('num_episodes')['id'].count().sort_values(ascending=False).head(30).plot(kind='bar', figsize=(15,10))
plt.show()

*Notes:* A lot of Movies (1 episode) that's why the spike, but the summation of all others are the other percentages of tv,ova,ona,... etc. most tv/specials are short 12 (episodes)/(season|title). </br>
*The fans of "When you have eliminated the impossible" teenager for 22+ years don't give up :(* </br>
*Gomu Gomu no guys don't be Sadge :(*

# 2- **MODELS TIME:** :3
![image info](https://hips.hearstapps.com/hmg-prod.s3.amazonaws.com/images/gettyimages-458406992-1538405221.jpg?crop=0.9xw:0.9xh;0,0&resize=256:*) <br />
  July's 2022 Work

## Similarity Analysis :

### Synopsis Keyword Analysis:
*(NLP)* :
* KeyBERT.
* Spacy.
* tqdm.
* CountVectorizer.
* TF-IDF

In [ ]:
import sweetviz as sv
#You could specify which variable in your dataset is the target for your model creation. We can specify it using the target_feat parameter.
data_report = sv.analyze(data)



In [ ]:
data_report.show_notebook(w=1500, h=900, scale=0.8)
data_report.show_html(scale=0.9)

### Cos-similarity:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet


In [ ]:
tfidf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['synopsis'] + data['genres'] + data['rating'] + data['studios']+data['media_type'])
tfidf_matrix.shape

Using the Cosine Similarity to calculate a numeric quantity that denotes the similarity between two movies. 

$cosine(x,y) = \frac{x. y^\intercal}{||x||.||y||}$

In [ ]:
cos_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
data = data_main.reset_index()
titles = data['title']
indices = pd.Series(data_main.index, index=data['title'])

In [ ]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cos_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    anime_indices = [i[0] for i in sim_scores]
    return titles.iloc[anime_indices]
data['title'][3]

In [ ]:
cos_results=get_recommendations('Death Note').head(10)
cos_results

Not so close recommendations but good start


#### **Featuring keywords and similarities:**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(tokenizer = lambda x: x.split(","), binary='true')
#vectors=[None]*len(keywords)

In [ ]:
dfsyn['keywords'].head(4)

In [ ]:
dfsyn.head()

In [ ]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

# Apply clean_data function to your features.
features = ['genres', 'keywords', 'rating', 'media_type']

for feature in features:
    dfsyn[feature] = dfsyn[feature].apply(clean_data)

In [ ]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['genres']) + ' ' + x['rating'] + ' ' + ' '.join(x['media_type'])
dfsyn['soup'] = dfsyn.apply(create_soup, axis=1)

In [ ]:
dfsyn['soup'].head()

In [ ]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cos_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cos_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    anime_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return dfsyn['title'].iloc[anime_indices]

In [ ]:


count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(dfsyn['soup'])



cos_sim2 = cosine_similarity(count_matrix, count_matrix)
dfsyn = dfsyn.reset_index()
indices = pd.Series(dfsyn.index, index=dfsyn['title'])

In [ ]:
cos2_results=get_recommendations('Death Note', cos_sim2)
len(cos2_results),cos2_results


**Cos-1 is genertated from similarity of multiple features using TF-IDF one of them was the whole synopsis of animes, while Cos2 was using keywords of synopsis instead of the whole synopsis feature**

In [ ]:
from keras.layers import Add, Activation, Lambda, BatchNormalization, Concatenate, Dropout, Input, Embedding, Dot, Reshape, Dense, Flatten

import keras
from keras import layers 
from keras.models import Model
from keras.optimizers import Adam

In [ ]:
rdf=pd.read_csv(r"../data-history/ratings-2020/rating_complete.csv")
n = 10

# Count the lines or use an upper bound
num_lines = sum(1 for l in open(r"../data-history/ratings-2020/rating_complete.csv"))

# The row indices to skip - make sure 0 is not included to keep the header!
skip_idx = [x for x in range(1, num_lines) if x % n != 0]

# Read the data
#rdf = pd.read_csv(r"../data-history/ratings-2020/rating_complete.csv", skiprows=skip_idx )
print(rdf.shape)
rdf.columns

### **Pair wise distance** :

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances

In [ ]:
# Removing Duplicated Rows
duplicates = rdf.duplicated()

if duplicates.sum() > 0:
    print('> {} duplicates'.format(duplicates.sum()))
    rdf = rdf[~duplicates]

print('> {} duplicates'.format(rdf.duplicated().sum()))

In [ ]:
rdf.columns

In [ ]:
# Scaling BTW (0 , 1.0)
min_rating = min(rdf['rating'])
max_rating = max(rdf['rating'])
rdf['rating'] = rdf["rating"].apply(lambda x: (x - min_rating) / (max_rating - min_rating)).values.astype(np.float64)

AvgRating = np.mean(rdf['rating'])
print('Avg', AvgRating)

In [ ]:
rdf.head(2)

In [ ]:
g = rdf.groupby('user_id')['rating'].count()
top_users = g.dropna().sort_values(ascending=False)[:20]
top_r = rdf.join(top_users, rsuffix='_r', how='inner', on='user_id')

g = rdf.groupby('id')['rating'].count()
top_animes = g.dropna().sort_values(ascending=False)[:20]
top_r = top_r.join(top_animes, rsuffix='_r', how='inner', on='id')

pd.crosstab(top_r.user_id, top_r.id, top_r.rating, aggfunc=np.sum)

In [ ]:
top_r.isnull().sum()

In [ ]:
anime_ids = rdf["id"].unique().tolist()
anime2anime_encoded = {x: i for i, x in enumerate(anime_ids)}
n_animes = len(anime2anime_encoded)
anime_encoded2anime = {i: x for i, x in enumerate(anime_ids)}

rdf["anime"] = rdf["id"].map(anime2anime_encoded)

user_ids = rdf["user_id"].unique().tolist()
user2user_encoded = {x: i for i, x in enumerate(user_ids)}
user_encoded2user = {i: x for i, x in enumerate(user_ids)}
n_users = len(user2user_encoded)
rdf["user"] = rdf["user_id"].map(user2user_encoded)


print("Num of users: {}, Num of animes: {}".format(n_users, n_animes))
print("Min rating: {}, Max rating: {}".format(min(rdf['rating']), max(rdf['rating'])))

In [ ]:
tmatrix = np.zeros((n_users, n_animes))
tmatrix.shape

In [ ]:
rdf.columns, rdf.shape

In [ ]:
for line in rdf.itertuples():
    tmatrix[line[5]-1, line[4]-1] = line[3]

In [ ]:
tmatrix[1]

In [ ]:
user_distances = pairwise_distances(tmatrix, metric="cosine")

# ".T" below is to transpose our 2D matrix.
tmatrix_transpose = tmatrix.T
anime_distances = pairwise_distances(tmatrix_transpose, metric="cosine")

user_distances.shape, anime_distances.shape

In [ ]:
user_similarity = 1 - user_distances
anime_similarity = 1 - anime_distances

In [ ]:
data_main.columns

In [ ]:
idx_to_anime = {}
for line in data_main.itertuples():
        idx_to_anime[(line[1])-1] = line[3]
anime_to_idx = {v: k for k, v in idx_to_anime.items()}

In [ ]:
anime_idx= anime_to_idx['Death Note']

def top_k_similar(similarity, mapper , anime_idx, k=8):
      return [mapper[x] for x in np.argsort(similarity[anime_idx,:])[:-k-2:-1]]

In [ ]:
pair_results= top_k_similar(anime_similarity,idx_to_anime ,anime_idx,k=8)
print(pair_results)

In [ ]:
def jaccard_similarity(a, b):
    # convert to set
    a = set(a)
    b = set(b)
    # calucate jaccard similarity
    j = float(len(a.intersection(b))) / len(a.union(b))
    return j
print('pair vs cos2')
print(jaccard_similarity(pair_results,cos2_results))
print('cos1 vs cos2')
print(jaccard_similarity(cos_results,cos2_results))

**zenzen wakaranaaaaaiiiii !!!!!!!!!!!!!** </br>
:"D </br>
pair-wise distance results not related to cosine Similarity results at all no intersections. </br>
using keywords or using Full synopsis didn't matter for cos similarity so better for resources use keywords

### **RecommendNet Maybe?** :

**Zenzen heiki janai :"D , Tasukete, Dare ka tasukeeteeeeee !** <br />
  Aug 2022 Work

In [ ]:
rdf.shape,rdf.isnull().sum(),len(rdf['user'])

In [ ]:
df.filter(regex='^media_type_',axis=1).head(2), df.filter(regex='^source_',axis=1).head(2)

In [ ]:
data['id'].values[1],data['popularity'].values[1]

#### *Normal Recommender features*

**Collaborative Filtering Approach, dependancy on user rates**

In [ ]:
# Shuffle
rdf = rdf.sample(frac=1, random_state=73)

X = rdf[['user', 'anime']].values
y = rdf["rating"]

In [ ]:
# Split
test_set_size = 10000 #10k for test set
train_indices = rdf.shape[0] - test_set_size 

X_train, X_test, y_train, y_test = (
    X[:train_indices],
    X[train_indices:],
    y[:train_indices],
    y[train_indices:],
)

print('> Train set ratings: {}'.format(len(y_train)))
print('> Test set ratings: {}'.format(len(y_test)))

In [ ]:
X_train_array = [X_train[:, 0], X_train[:, 1]]
X_test_array = [X_test[:, 0], X_test[:, 1]]

In [ ]:
print(len(X_train))
print(len(y_train))

In [ ]:
X_train_array = [X_train[:, 0], X_train[:, 1]]
X_test_array = [X_test[:, 0], X_test[:, 1]]

In [ ]:
print(len(X_train_array[0]))
print(len(X_test_array[0]))

In [ ]:
def recommender_net():
    embedding_size = 64
    
    user = Input(name = 'user', shape = [1])
    user_embedding = Embedding(name = 'user_embedding',
                    input_dim = n_users, 
                    output_dim = embedding_size)(user)
    
    anime = Input(name = 'anime', shape = [1])
    anime_embedding = Embedding(name = 'anime_embedding',
                    input_dim = n_animes, 
                    output_dim = embedding_size)(anime)
    #x = Concatenate()([user_embedding, anime_embedding])
    x = Dot(name = 'dot_product', normalize = True, axes = 2)([user_embedding, anime_embedding])
    x = Flatten()(x)
    x = Dense(1, kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = Activation("sigmoid")(x)
    model = Model(inputs=[user, anime], outputs=x)
    model.compile(loss='binary_crossentropy', metrics=["mae", "mse"], optimizer='adam')
    
    return model

model1 = recommender_net()
model1.summary()


In [ ]:
# Callbacks
from tensorflow.python.keras.callbacks import Callback, ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping, ReduceLROnPlateau

start_lr = 0.00001
min_lr = 0.00001
max_lr = 0.00005
batch_size = 10000
rampup_epochs = 5
sustain_epochs = 0
exp_decay = .8

def lrfn(epoch):
    if epoch < rampup_epochs:
        return (max_lr - start_lr)/rampup_epochs * epoch + start_lr
    elif epoch < rampup_epochs + sustain_epochs:
        return max_lr
    else:
        return (max_lr - min_lr) * exp_decay**(epoch-rampup_epochs-sustain_epochs) + min_lr


lr_callback = LearningRateScheduler(lambda epoch: lrfn(epoch), verbose=0)

checkpoint_filepath = './weights.h5'

model_checkpoints = ModelCheckpoint(filepath=checkpoint_filepath,
                            save_weights_only=True,
                            monitor='val_loss',
                            mode='min',
                            save_best_only=True)

early_stopping = EarlyStopping(patience = 3, monitor='val_loss', 
                            mode='min', restore_best_weights=True)

my_callbacks = [
    model_checkpoints,
    lr_callback,
    early_stopping,   
]

In [ ]:
print(len(X_test_array[0]))
print(len(y_test))


In [ ]:
# Model training
history = model1.fit(
    x=X_train_array,
    y=y_train,
    batch_size=batch_size,
    epochs=20,
    verbose=1,
    validation_data=(X_test_array, y_test),
    callbacks=my_callbacks
)

model1.load_weights(checkpoint_filepath)

In [ ]:
#Training results
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(history.history["loss"][0:-2])
plt.plot(history.history["val_loss"][0:-2])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.show()

In [ ]:
from tqdm.keras import TqdmCallback


history = model1.fit(
    x=X_train_array,
    y=y_train,
    batch_size=batch_size,
    epochs=30,
    validation_data=(X_test_array, y_test),
    verbose = 0, 
    callbacks=[TqdmCallback(verbose=0)])

model1.load_weights(checkpoint_filepath)



In [ ]:
#Training results
import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(history.history["loss"][0:-2])
plt.plot(history.history["val_loss"][0:-2])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.show()

In [ ]:
def extract_weights(name, model):
    weight_layer = model.get_layer(name)
    weights = weight_layer.get_weights()[0]
    weights = weights / np.linalg.norm(weights, axis = 1).reshape((-1, 1))
    return weights

anime_weights = extract_weights('anime_embedding', model1)
user_weights = extract_weights('user_embedding', model1)

In [ ]:
data_main.columns

In [ ]:
name = data[data_main.id == 100].title.values[0]
print(name)

In [ ]:
# Fixing Names
def get_animename(anime_id):
    try:
        name = data[data_main.id == anime_id].title.values[0]
        return name
    except:
        print('error')
        return 0

data["eng_version"] = data['title']


data_main.sort_values(by=['mean'], 
                inplace=True,
                ascending=False, 
                kind='quicksort',
                na_position='last')

df = data[["id","title", "mean", "genres", "num_episodes", 
        "media_type","synopsis"]]


def get_animeframe(anime):
    if isinstance(anime, int):
        return df[df.id == anime]
    if isinstance(anime, str):
        return df[df.title == anime]
def get_sypnopsis(anime):
    if isinstance(anime, int):
        return df[df.id == anime].synopsis.values[0]

In [ ]:
df.shape

In [ ]:
pd.set_option("max_colwidth", None)

def find_similar_animes(name, n, return_dist=False, neg=False):
        index = get_animeframe(name).id.values[0]
        print(index)
        encoded_index = anime2anime_encoded.get(index)
        weights = anime_weights
        print(encoded_index)
        dists = np.dot(weights, weights[encoded_index])
        sorted_dists = np.argsort(dists)
        
        n = n + 1            
        
        if neg:
            closest = sorted_dists[:n]
        else:
            closest = sorted_dists[-n:]
        print('animes closest to {}'.format(name))
        if return_dist:
            return dists, closest
        rindex = df
        similarityarr = []
        for close in closest:
            decoded_id = anime_encoded2anime.get(close)
            sypnopsis = get_sypnopsis(decoded_id)
            anime_frame = get_animeframe(decoded_id)
            anime_name = anime_frame.title.values[0]
            genre = anime_frame.genres.values[0]
            similarity = dists[close]
            similarityarr.append({"id": decoded_id, "title": anime_name,
                            "similarity": similarity,"genres": genre,
                            'synopsis': sypnopsis})
        frame = pd.Dataframe(similarityarr).sort_values(by="similarity", ascending=False)
        return frame[frame.id != index].drop(['id'], axis=1)


In [ ]:
find_similar_animes('Death Note', n=10, neg=False)

#### *Features modding* <br />
   Modifying parameters for Recommend NET

In [ ]:
# dfdl =pd.DataFrame()

In [ ]:
# dfdl_ids = data["id"].tolist()
# dfdlid_encoded = {x: i for i, x in enumerate(dfdl_ids)}
# n_animes = len(dfdlid_encoded)
# id_encoded2id = {i: x for i, x in enumerate(dfdl_ids)}
# dfdl["id"] = data["id"].map(dfdlid_encoded)

# dfdl_mean = data["mean"].tolist()
# dfdl_mean_encoded = {x: i for i, x in enumerate(dfdl_mean)}
# mean_encoded2mean = {i: x for i, x in enumerate(dfdl_mean)}
# n_users = len(dfdl_mean_encoded)
# dfdl["mean"] = data["mean"].map(dfdl_mean_encoded)

# dfdl_pop = data["popularity"].tolist()
# user2user_encoded = {x: i for i, x in enumerate(dfdl_pop)}
# user_encoded2user = {i: x for i, x in enumerate(dfdl_pop)}
# n_users = len(user2user_encoded)
# dfdl["popularity"] = data["popularity"].map(user2user_encoded)

In [ ]:
data_main.columns, data_main.shape

In [ ]:

# x1 = rdf[['user', 'anime']].values 

# #x2=  data[['id'],['popularity']].values
# x3=data[['mean'],['num_scoring_users']].values
# x4=data['rank'].tolist(),data['num_favorites'].tolist()
# x5= df.filter(regex='^media_type_',axis=1).values[i]
# x6= df.filter(regex='^source_',axis=1).values[i]

# y = rdf["rating"]
# # Split
# test_set_size = 250000 #10k for test set
# train_indices = rdf_sampled.shape[0] - test_set_size 
# len(x1),len(x2),len(x2[1]),len(x3),len(x3[1]),len(x4),len(x4[1]),len(y),


In [ ]:
# X3= x3[:,0] + x3[:,2] +x3[:,3] + x3[:,4] + x3[:,5] + x3[:,1] 
# X4=['None']*len(x4)*len(x4[1])
# for i in range(len(x4[1])):
#     X4 =X4 + x4[:,i]

In [ ]:
# X1_train, X1_test, y_train, y_test = (
#     x1[:train_indices],
#     x1[train_indices:],
#     y[:train_indices],
#     y[train_indices:],
# )

### *After Reading Some Articels:*

#### **Research at home** <br />
   Dec. 2022 work <br />

Semantic Similarity on synopsis using nlp models.

Potential Models for learning: <br />
* paraphrase-miniLM
* stsb-roberta latest alternatives
* bert-base-nli-mean-tokens

**To_Do:**
- Get embeddings from pretrained for all synopsis ( all paragraphs ).
- Compare Similarity using distance wise / cosine / pairwise whatever the hell will measure similarity of embeddings.
- Worst case senario ,(For each sentence embeddings in the requested anime synopsis loop cosine similarity between all sentences in all other synopsis)
- Optimization worth testing: Finding similarity between sentences in the same synopsis to get unique sentences and store those while ignoring sentences that are pretty much similar in embeddings, that leads to having smaller group of sentences for each synopsis to loop on (Still looping bratan).
- 5Head IDEA: Semantic Keyword embeddings similarity analysis to get potential chosen titles to do semantic sentence analysis on.
- **OR JUST USE PARAPHRASE MINING U Fokin IDIOT, anata BAKA ??? hontoni BAKAAAA.**